# Add Dependences

In [10]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [11]:
parameters = Model(
    Cubic(
        μ₁      = 0.0005,
        rₘᵢₙ    = 2,
        rₘₐₓ    = 3
    ),
    Contractile(
        fₚ       = 0.001
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 100,
        nn       = 100
    ),
    Geometry(
        R_agg    = 9,
        position = [
            1.0    0.0    0.0;
            0.0    1.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "Fusion.xyz"
    )
)

Model(Cubic(0.0005, 2.0, 3.0), Contractile(0.001), Time(100000.0, 0.1), Neighbor(100, 100), Geometry(9.0, [1.0 0.0 0.0; 0.0 1.0 0.0]), Simulation(200, "Fusion.xyz"))

In [12]:
# Variables for iterator
it_force = Cubic(
    μ₁      = 0.1,
    rₘᵢₙ    = 2,
    rₘₐₓ    = 3
)
it_contractile = Contractile(
    fₚ      = 0.0000001
)

## Var before Iterator
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(it_force::Cubic, r)


 Loading Force Function
  0.000049 seconds (17 CPU allocations: 1.227 KiB)


force (generic function with 1 method)

## Starting Data

In [14]:
# Generating Struct
CUDA.@time Agg = (
    Aggregate(
        Neighbor(50, 12), 
        PositionCell(
            # Loading Data
            CUDA.@time  X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
        )
    )
);

  0.009002 seconds (8.06 k CPU allocations: 317.703 KiB) (1 GPU allocation: 6.305 KiB, 0.33% memmgmt time)
  0.280172 seconds (353.68 k CPU allocations: 21.009 MiB) (13 GPU allocations: 4.819 MiB, 0.13% memmgmt time)


## run_event -> one_aggregate

In [15]:
# # Simulation One Aggregate
# CUDA.@time one_aggregate(Path*"/"*File, true, parameters, it_contractile, Agg)

## run_event -> fusion

In [16]:
# Simulating Two Aggregate
CUDA.@time fusion(Path*"/"*File, true, parameters, it_contractile, Agg)

Finding Equilibrium in one Aggregate


Progress:   2%|▋                        |  ETA: 1:19:16m46m

LoadError: InterruptException: